In [1]:
#!/usr/bin/env python3
import qiskit
from qiskit import (
    QuantumCircuit,
    QuantumRegister,
    ClassicalRegister,
    transpile
)
from qiskit.quantum_info import (
    Kraus, 
    SuperOp,
)
from qiskit_aer.noise import(
    depolarizing_error,
    phase_damping_error,
    NoiseModel,
    QuantumError,
    amplitude_damping_error,
)
# from qiskit_ibm_provider import IBMProvider
from qiskit import Aer
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import math
import sys
sys.path.append('../')
import utilities as utl
import importlib
import json
importlib.reload(utl)

def heralded_circ():
    '''raw entanglment swapping.'''
    qr_a=QuantumRegister(2, "alice")
    # qr_c=QuantumRegister(4, "charlie")
    qr_b=QuantumRegister(2, "bob")
    cr_epr=ClassicalRegister(2, "cr_epr")
    cr_h=ClassicalRegister(2, "cr_heralded")

    qc=QuantumCircuit(qr_a, qr_b, cr_epr, cr_h)
    # Bell States
    qc.h(qr_a[1])
    qc.cx(qr_a[1], qr_a[0])
    qc.h(qr_b[1])
    qc.cx(qr_b[1], qr_b[0])
    # Channel
    qc.id(qr_a)
    qc.id(qr_b)
    # Entanglement swap
    qc.cx(qr_a[0], qr_b[0])
    qc.h(qr_a[0])
    qc.barrier()
    qc.measure(qr_a[0], cr_h[0])
    qc.measure(qr_b[0], cr_h[1])
    bsm_ancillas=[qr_a[0], qr_b[0]]
    utl._add_fixing_gates(qc, qr_a, cr_h, 1)
    return qc, qr_a, qr_b, cr_epr


In [2]:
SHOTS=10000
rawFidelities=[]
errors=np.linspace(0.0, stop=0.5, num=11, endpoint=True)
#no pcs
qc, qr_a, qr_b, cr_epr = heralded_circ()
error_gate1=0.001
for idx, error in enumerate(errors):
        print(f"depolarizing error: {error}")
        noise_model=utl.get_noise_model(error_gate1)
        errorPChannel=depolarizing_error(error, 1)
        noise_model.add_all_qubit_quantum_error(errorPChannel, ["id"])
        print(noise_model)
        backend=AerSimulator(noise_model=noise_model)
       
        # print(qc)
        rawResultxx=utl.meas_xx(qc, [qr_a[1], qr_b[1]], cr_epr, backend, SHOTS)
        rawResultyy=utl.meas_yy(qc, [qr_a[1], qr_b[1]], cr_epr, backend, SHOTS)
        rawResultzz=utl.meas_zz(qc, [qr_a[1], qr_b[1]], cr_epr, backend, SHOTS)

        epr_idx=-1 #the position of the classical registers that contain measurements of the 
                #epr pair.
        rawExpectxx=utl.expect_of_vals(utl.to_percent(utl.collect_meas(rawResultxx.get_counts(), epr_idx)))
        rawExpectyy=utl.expect_of_vals(utl.to_percent(utl.collect_meas(rawResultyy.get_counts(), epr_idx)))
        rawExpectzz=utl.expect_of_vals(utl.to_percent(utl.collect_meas(rawResultzz.get_counts(), epr_idx)))
        rawFidelity=utl.fidelity_phip(rawExpectxx, rawExpectyy, rawExpectzz)
        rawFidelities.append(rawFidelity)
        print(f"noisy no pcs fidelity: {rawFidelity}")
print(f"raw fidelities: {rawFidelities}")
with open("json\\raw_unbiased.json", "w") as file:
        json.dump({"errors": errors.tolist(),"fidelities": rawFidelities}, file)


depolarizing error: 0.0
NoiseModel:
  Basis gates: ['ccx', 'cx', 'cz', 'h', 'id', 'rz', 'sx', 'x', 'y', 'z']
  Instructions with noise: ['y', 'x', 'rz', 'h', 'z', 'ccx', 'cx', 'cz', 'measure']
  All-qubits errors: ['cx', 'cz', 'ccx', 'h', 'x', 'y', 'z', 'rz', 'measure']
xx circ: 
                    ┌───┐┌───┐     ┌───┐ ░ ┌─┐    ░                           »
      alice_0: ─────┤ X ├┤ I ├──■──┤ H ├─░─┤M├────░───────────────────────────»
               ┌───┐└─┬─┘├───┤  │  └───┘ ░ └╥┘    ░ ┌────────── ┌──────────── »
      alice_1: ┤ H ├──■──┤ I ├──┼────────░──╫─────░─┤ Switch-0  ┤ Case-0 (1)  »
               └───┘┌───┐├───┤┌─┴─┐      ░  ║ ┌─┐ ░ └────╥───── └──────────── »
        bob_0: ─────┤ X ├┤ I ├┤ X ├──────░──╫─┤M├─░──────╫────────────────────»
               ┌───┐└─┬─┘├───┤└───┘      ░  ║ └╥┘ ░      ║                    »
        bob_1: ┤ H ├──■──┤ I ├───────────░──╫──╫──░──────╫────────────────────»
               └───┘     └───┘           ░  ║  ║  ░      ║                    »